In [7]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

# `range` ושימושיו בלולאת `for`

### הטיפוס `range`

ציינו שלולאות `for` מסוגלות לעבור על רשימות ועל מחרוזות. מה משותף לשני הטיפוסים?
שני הטיפוסים הללו מייצגים סדרה כלשהיא של ערכים, ומאפשרים מעבר על האיברים שלהם בזה אחר זה.
טיפוסים מסוג אלו נקראים **איטרביליים** (iterables). 
לולאת `for` יודעת לעבוד בדיוק עם אובייקטים כאלה: היא "שולפת" מהם כל איבר בתורו, עד שנגמרים הערכים.

נציג כעת את `range()` - פונקציה המאפשרת לנו ליצור אובייקט איטרבילי נוסף: **סדרה ממוינת של מספרים שלמים בתחום מסוים**.  
התוצאה היא אובייקט מטיפוס `range` שמייצר את המספרים לפי דרישה, כלומר המספרים נוצרים רק כאשר אנחנו צריכים אותם, ולא נשמרים בזיכרון כקבוצה שלמה.

זה מאוד שימושי כשמייצרים סדרות מספרים או כאשר רוצים לעבור בלולאת `for` על רשימות לפי אינדקסים.

#### תחביר

טווח פשוט מוגדר ע"י `range(stop)`, המייצר מספרים מאפס (כולל) עד stop (לא כולל) בקפיצות של 1. כלומר, כשאין step וגם start, מניחים שמתחילים מ0 והולכים בקפיצות של 1.


בנוסף, ניתן גם לייצר טווחים יותר מורכבים.  
בדומה לslicing שראינו ברשימה, גם ב`range` מגדירים את תת הקבוצה המבוקשת (מתוך קבוצת המספרים השלמים) לפי start, stop ו-step:
- `range(start, stop, step)` - מייצר מספרים מstart (כולל) עד stop (לא כולל) בקפיצות של step.
- `range(start, stop)`  - מייצר מספרים מstart (כולל) עד stop (לא כולל) בקפיצות של 1. כלומר, כשאין step, מניחים שמדובר בקפיצות של 1.

כדי לראות את הערכים ש`range` מייצר, נצטרך להמיר אותו לרשימה.

In [3]:
print(list(range(10)))
print(list(range(3,10, 2)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[3, 5, 7, 9]


```{admonition} **שימו לב**
:class:  error
המרת `range` לרשימה נעשית כאן רק לצרכי בדיקת הקוד שלנו. בד"כ, נשתדל להימנע מהמרה כזו, כיוון שהיא למעשה תשמור את כלל המספרים בזכרון.  
זכרו שאחד הכוחות של `range` הוא ליצור את המספר הבא בסדרה לפי דרישה, ולא את כל המספרים בבת אחת.
```

```{admonition} **שימו לב**
:class:  error
- בדומה לslicing ברשימות, `step` יכול להיות גם שלילי - אבל אז נצטרך שיתקיים `start>end`.
- `step` חייב להיות שונה מ0.
```


### לולאות `for` על `range`

בפרק הקודם ראינו איך משתמשים ב`range` כדי לייצג סדרה של מספרים.  בדומה לאובייקטים מטיפוסים `str` ו`list`, אפשר לעבור גם על איבריו של `range` באמצעות לולאת `for`, בדיוק כפי שעשינו עם רשימות ומחרוזות. בחלק זה נראה כיצד.

הדוגמה שתלווה אותנו כאן היא הבעיה הבאה: אנחנו רוצים לחשב את סכום המספרים השלמים בין 1 לבין מיליון. חלקכם אולי יודעים שיש לזה נוסחה, וגם נציג אותה בהמשך, אבל כרגע "נשכח" ממנה כדי להדגים איך משתמשים בלולאות כדי לפתור את הבעיה.

#### חישוב סכום של טווח מספרים

אילו רצינו לחשב את סכום המספרים השלמים בין 1 ל-10, זה לא היה דורש שימוש בלולאות בכלל. פשוט כותבים בפייתון: 
```python
print(1+2+3+4+5+6+7+8+9+10)
```
ומקבלים את התוצאה (55 אגב, למי שלא יכול לחכות...).

אבל לא סביר לפעול בגישה הזו לחישוב סכום המספרים בין 1 למיליון, אלא אם ממש משעמם לכם ואתם לעולם לא מתעייפים... 

```{admonition} **עצרו וחשבו:** האם הפונקציה המובנית sum של פייתון לא מספיקה כדי לפתור את בעית סכימה המספרים מאחד למליון בקלות?
:class: dropdown, caution

לא. אמנם הפונקציה `sum` של פייתון מאפשרת לחשב בפקודה אחת סכום של רשימת מספרים, אבל איך נבנה רשימה כזאת? לכתוב lst = [1, 2, 3, ..., 1000000] יהיה מאוד מסורבל ולא יעיל. פתרון טוב יותר יהיה להשתמש ב-iterables (כמו `range`), שמייצג את המספרים מבלי לכתוב אותם במפורש ולשמור בזכרון.

```

כעת נדגים בקוד הבא שימוש ב`range` עם לולאת `for`.

In [3]:
# Simple for loop over range
for num in range(1, 3):
	print(num)


1
2


תזכורת: כאשר משתמשים ב`range`, כמו שראינו בslicing, המספר האחרון אינו כלול בטווח, ולכן לא הודפס לנו המספר 3.

איזה מספר יודפס בסוף ריצת מקטע הקוד שלפניך? 
```python
p = 1
for num in range(1, 5): 
    p = p * num
print(p)
```

In [3]:
import json
from jupyterquiz import display_quiz

example = '''
[
  {
    "question": "",
    "type": "numeric",
    "answers": [
      {
        "type": "value",
        "value": 24,
        "correct": true,
        "feedback": "נכון!"
      },
      {
        "type": "default",
        "feedback": "נסו שוב. טיפ: מדובר במכפלה של המספרים 1 עד 4."
      }
    ]
  }
]
'''
myquiz = json.loads(example)
display_quiz(myquiz)


<IPython.core.display.Javascript object>

מכאן הדרך קצרה לכתיבת תוכנית שמחשבת סכום של סדרת המספרים השלמים בין 1 למליון (או `10**6`). בפסאודו קוד זה ייראה כך:


1. הגדר משתנה סכום בשם s ואתחל אותו באפס
2. לכל מספר בטווח בין 1 לבין `10**6`, נסמנו num:  
      2.1 הגדל את s ב-num
3. הדפס את s.

ובפייתון זה ייראה כך (הריצו ובידקו מהו הסכום המתקבל):

In [45]:
# Simple for loop summing over range
s = 0
for num in range(1, 10**6+1):
    s = s+num
print(s)


500000500000


```{admonition} **עצרו וחשבו:**
:class: caution

למה הגדרנו את הטווח על-ידי `10**6 + 1` ולא פשוט עד מיליון?

```


התוכנית הזאת עובדת מצויין. אבל דמיינו שאתם כותבים תוכנית ארוכה, ובכמה מקומות בתוכנית שלכם צריך לחשב סכום של טווח מספרים, טווח אחר בכל פעם. במקום להעתיק את 4 השורות האלו בכל פעם ולשנות את הגדרת הטווח בהתאם, אפשר להגדיר פונקציה פעם אחת ולקרוא לה בכל פעם שצריך לחשב סכום של טווח. מכיוון שהטווח עלול להיות שונה בכל פעם, אפשר להגדיר שהפונקציה תקבל שני פרמטרים – תחילת הטווח וסוף הטווח, ותחשב את הסכום בהתאם. בפייתון זה ייראה כך:
```python
def sum_range(start, end):
    s = 0
    for num in range(start, end+1):
        s = s+num
    return s
```

הריצו את הקוד הבא וצפו בתוצאה:

In [47]:

# Executions of sum_range
def sum_range(start, end):
    s = 0
    for num in range(start, end+1):
        s = s+num
    return s


res = sum_range(1, 10**6)
print('The sum of all the integers between 1 and 1,000,000 is', res)
print('But the sum of all the integers between 3 and 3756 is', sum_range(3, 3756))
print('And the sum of all the integers between -45 and 45 is', sum_range(-45, 45))

The sum of all the integers between 1 and 1,000,000 is 500000500000
But the sum of all the integers between 3 and 3756 is 7055643
And the sum of all the integers between -45 and 45 is 0


```{admonition} **שימו לב**
:class: error

הפונקציה מחזירה (`return`) את התוצאה, ולא מדפיסה אותה. כשקוראים לפונקציה בדרך כלל עדיף להחזיר את הערך, ולהדפיס את הערך שחזר, במקום להדפיס בתוך הפונקציה עצמה. כך ניתן יהיה להשתמש בערך שחזר גם לדבר נוספים מחוץ לפונקציה, מעבר להדפסה בלבד.

```

אחרי שעברנו את כל הדרך לכתיבת פונקציה שמחשבת סכום של טווח מספרים, נחזור לרגע לפונקציה המובנית `sum` של פייתון. 
מסתבר **שניתן להפעיל את `sum` על טווח מספרים שמוגדר על-ידי `range`**. במילים אחרות יכולנו להחליף את הלולאה שכתבנו קודם בפקודה אחת:

```python
def sum_range2(start, end):
    return sum(range(start, end+1))
```

כיצד הצלחנו לחשב את הסכום הנדרש בשורה בודדת? מאחורי הקלעים, **מימוש הפונקציה `sum` עוברת איבר-איבר על איברי הטווח** ועושה חישוב מאוד דומה לחישוב שעשינו קודם באמצעות לולאה. עם זאת, **לרוב, פונקציות מובנות של פייתון עובדות מהר יותר מאשר מימושים שלנו**. ובאמת, אפשר לראות הבדלים ניכרים בזמן הריצה של `sum_range` שכתבנו קודם ל-`sum_range2` הזו.   
ההבדלים זניחים על טווח מספרים קטן, אבל ככל שהטווח גדול יותר ההבדלים נעשים בולטים ומשמעותיים יותר. הריצו למשל את הקוד בחלונית הבאה, שמודד זמני ריצה של חישוב הסכום של הטווח בין 1 ל-`10**8`, כלומר מאה מיליון. אל תתעכבו על פרטי המימוש של מדידת הזמנים כרגע. נחזור לזה בהמשך השיעור. 

לאחר ההרצה שנו את הקריאה `res = sum_range(1, 10**8)` לקריאה `res = sum_range2(1, 10**8)` והריצו שוב. 

In [28]:
# Time measurement. You can ignore this code block
import time
def timing(foo, *args):
    start=time.perf_counter()
    res = foo(*args)
    end = time.perf_counter()
    exec_time=round(end-start,2)
    print(f'Execution of foo.__name__ took {exec_time} seconds. Returned value: {res}')

In [29]:
# Time comparison for summing over range
def sum_range(start, end):
    s = 0
    for num in range(start, end+1):
        s = s+num
    return s

def sum_range2(start, end):
    return sum(range(start, end+1))

# This code timing 
timing(sum_range, 1 , 10**8)
timing(sum_range2, 1 , 10**8)

Execution of foo.__name__ took 5.66 seconds. Returned value: 5000000050000000
Execution of foo.__name__ took 2.07 seconds. Returned value: 5000000050000000


```{admonition} **מעניין לדעת!**  קיימת נוסחה פשוטה לחישוב סכום מהצורה הזו.
:class: dropdown, note

יש נוסחה פשוטה לחישוב סכום של סדרה מהצורה 1+2+...+n עבור n שלם חיובי כלשהו.  
את הנוסחה גילה המתמטיקאי הנודע פרידריך גאוס, ככל הנראה בהיותו בן 6 בלבד!

$$
1+2+3+\dots+n = \frac{n(n+1)}{2}
$$

למשל, לפי נוסחה זו הסכום של המספרים השלמים בין 1 ל-10 הוא:

$$
1+2+3+\dots+10 = \frac{10(10+1)}{2} = \frac{110}{2} = 55
$$

והסכום של $1+2+\dots+10^8$ שווה לפי נוסחה זו ל:

$$
\frac{10^8 \cdot (10^8+1)}{2}
$$

חשבו כיצד לממש זאת בקוד!  
רמז: האם עדיף להשתמש ב`/` או ב`//`? 

חישוב כזה יעיל הרבה יותר מאשר שימוש בלולאה, ואפילו יותר מאשר שימוש בפונקציה המובנית `sum` של פייתון (שכאמור בעצמה עוברת על כל איברי הרשימה בלולאה). 
כדי להיווכח עד כמה חישוב זה יעיל יותר, החליפו את שורת הקריאה ל-sum_range בחלונית שלמעלה במימוש החדש והריצו שוב.

אומנם התוצאה נראת דומה למדי לתוצאת ההרצה הקודמת, אבל למעשה מדובר במספר קטן הרבה יותר. רוב הסיכויים שתוצאת ההרצה תוצג כאשר בסופה האות e, אחריה מקף ואז מספר כמו 06. לדוגמה: 2.38102294921875e-06 **המספר הזה הוא לא 2.38 שניות, אלא 2.38 מיליוניות השניה**. בעצם הסיומת e-06 משמעותה שיש לכפול את המספר בעשר בחזקת מינוס 6, שזה 1 חלקי מיליון. מספר ממש קטן.
```

#### תרגול לולאות עם `range`

צרו לולאה שמדפיסה את כל המספרים המתחלקים ב-3 מ30 (כולל) עד 102 (כולל).
עליכם לממש את הקוד בשתי שורות בלבד!

In [7]:
# Write your code here

ממשו את הפונקציה `countdown` אשר המקבלת מספר שלם חיובי n, ומדפיסה ספירה לאחור מהמספר n כולל עד 1 (כולל).
עליכם לממש את תוכן הפונקציה בשתי שורות בלבד!

In [20]:
def countdown(n):
    # Write your code here
    pass
    

עכשיו נשים לב למשהו מאוד שימושי -  כאשר כותבים `range` עם פרמטר יחיד בתוך הסוגריים (למשל range(100)), הטווח מתחיל ב-0, קופץ ב-1 כל פעם, והפרמטר היחיד מציין את סוף הטווח. range(100) אם כן הוא טווח המספרים בין 0 ל-99 בקפיצות של 1. כלומר 100 מספרים שונים.

מכאן, שישנה דרך מאוד פשוטה בפייתון לומר "עשה 100 פעמים את הפעולה X":
```python
for i in range(100):
     Do X
```
ובאופן כללי, כדי לבצע פעולה מסויימת n פעמים:
```python
for i in range(n):
     Do X
```
בעצם עוברים פה על טווח המספרים `0, 1, ..., n-1` כלומר מבצעים בדיוק n איטרציות, ובכל אחת מהן מבצעים את הפעולה X. הפעולה X  יכולה להשתמש בערכו של i, אבל כמובן לא חייבת. למשל:
```python
for i in range(100):
     print(i)
```
לעומת:
```python
for i in range(100):
     print("hello")
```

לסיכום - ניתן להשתמש ב`range` כדי לעבור על סדרות מספרים בלולאת `for`. המשתנה שמקבל את כל איבר בסדרה משמש בדרך כלל בתוך הלולאה, אך אין חובה להשתמש בו.

### תרגול

לפניכם קטע קוד קצר:

``` python
x = 0
y = 0
z = 0

for num in range(10, 20, 3):
    if num > 5:
        x = x+1
    if num == 3:
        y = y+1
    else:
        z = z+2
```

In [67]:
import json
from jupyterquiz import display_quiz

example = '''
[
  {
    "question": "מה הערך הסופי של x לאחר ביצוע הקוד? (x מתחיל מ-0)",
    "type": "numeric",
    "answers": [
      {
        "type": "value",
        "value": 4,
        "correct": true,
        "feedback": "נכון! הלולאה רצה 4 פעמים, וכל פעם num > 5."
      },
      {
        "type": "default",
        "feedback": "נסה לעקוב אחרי תנאי if num > 5."
      }
    ]
  },
  {
    "question": "מה הערך הסופי של y לאחר ביצוע הקוד? (y מתחיל מ-0)",
    "type": "numeric",
    "answers": [
      {
        "type": "value",
        "value": 0,
        "correct": true,
        "feedback": "נכון! num לעולם לא שווה ל-3, ולכן התנאי לא מתקיים."
      },
      {
        "type": "default",
        "feedback": "נסה לבדוק אם num == 3 אי פעם מתקיים."
      }
    ]
  },
  {
    "question": "מה הערך הסופי של z לאחר ביצוע הקוד? (z מתחיל מ-0)",
    "type": "numeric",
    "answers": [
      {
        "type": "value",
        "value": 8,
        "correct": true,
        "feedback": "נכון! התנאי else מתבצע כל פעם ולכן z += 2 רץ 4 פעמים."
      },
      {
        "type": "default",
        "feedback": "בדוק כמה פעמים מתבצע else (כאשר num ≠ 3)."
      }
    ]
  }
]
'''
myquiz = json.loads(example)
display_quiz(myquiz)


<IPython.core.display.Javascript object>

לפניכם קטע קוד קצר:

```python
for i in range(10, 20, -1):
    print("Hello!")
```

In [70]:
import json
from jupyterquiz import display_quiz

example = '''
[
  {
    "question": "כמה איטרציות תבצע לולאת ה-for בקוד לעיל?",
    "type": "numeric",
    "answers": [
      {
        "type": "value",
        "value": 0,
        "correct": true,
        "feedback": "נכון! range(10, 20, -1) לא מפיקה אף ערך כי הכיוון הפוך."
      },
      {
        "type": "default",
        "feedback": "נסה לבדוק אם הכיוון של הטווח (מה-10 עד 20 עם צעד שלילי) אפשרי."
      }
    ]
  }
]
'''
myquiz = json.loads(example)
display_quiz(myquiz)


<IPython.core.display.Javascript object>

### מעבר על אינדקסים של רשימה או מחרוזת

עוד דרך שימושית להיעזר ב`range` בלולאות `for` היא לבצע איטרציה על ה**אינדקסים** של רשימה או מחרוזת נתונה. בואו נראה זאת:

In [1]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/QbCIn1Mw4QE" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

**לנוחיותכם חלונית עם הקוד שראינו בסרטון. מוזמנים להריץ בעצמכם:**


In [1]:
%%html
<iframe width="800" height="500" frameborder="0" src="https://pythontutor.com/iframe-embed.html#code=grades_lst%20%3D%20%5B90,%20100,%2085,%2085,%2075,%2095%5D%0A%0A%23%20for%20grade%20in%20grades_lst%3A%0A%23%20%20%20%20%20print%28grade%29%0An%20%3D%20len%28grades_lst%29%0Afor%20i%20in%20range%28n%29%3A%0A%20%20%20%20print%28grades_lst%5Bi%5D%29&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=true&origin=opt-frontend.js&py=311&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>

לפניכם הקוד הבא:

``` python
def what(lst):
    n = len(lst)
    for i in range(n):
        if lst[i] > 0:
            print(i)

what([3, -1, 6, 6, -9, 0, 1])
```

In [4]:
import json
from jupyterquiz import display_quiz

example = '''
[
  {
    "question": "מה המספר הראשון שמודפס על ידי הפונקציה what?",
    "type": "numeric",
    "answers": [
      {
        "type": "value",
        "value": 0,
        "correct": true,
        "feedback": "נכון! האיבר הראשון (3) הוא חיובי ולכן יודפס i=0."
      },
      {
        "type": "default",
        "feedback": "נסה לבדוק מהו הערך הראשון החיובי ברשימה."
      }
    ]
  },
  {
    "question": "מה המספר השני שמודפס על ידי הפונקציה what?",
    "type": "numeric",
    "answers": [
      {
        "type": "value",
        "value": 2,
        "correct": true,
        "feedback": "נכון! האיבר החיובי הבא נמצא באינדקס 2."
      },
      {
        "type": "default",
        "feedback": "נסה לעקוב אחרי הלולאה ולזהות מתי lst[i] > 0."
      }
    ]
  },
  {
    "question": "מה המספר השלישי שמודפס על ידי הפונקציה what?",
    "type": "numeric",
    "answers": [
      {
        "type": "value",
        "value": 3,
        "correct": true,
        "feedback": "נכון! זהו אינדקס נוסף של ערך חיובי (6)."
      },
      {
        "type": "default",
        "feedback": "בדוק את הערכים ברשימה לפי הסדר."
      }
    ]
  },
  {
    "question": "מה המספר הרביעי שמודפס על ידי הפונקציה what?",
    "type": "numeric",
    "answers": [
      {
        "type": "value",
        "value": 6,
        "correct": true,
        "feedback": "נכון! הערך החיובי האחרון הוא באינדקס 6 (המספר 1)."
      },
      {
        "type": "default",
        "feedback": "עבור כל איבר ברשימה, בדוק האם הוא חיובי והוסף את האינדקס שלו."
      }
    ]
  }
]
'''
myquiz = json.loads(example)
display_quiz(myquiz)


<IPython.core.display.Javascript object>